# Casusopdracht - Films

**Klas:** V2B
**Studenten:** 
- Roan Gaasbeek
- Mathias Hendriks
- Luc Pikaar


In [ ]:
import numpy as np
import pandas as pd

# 1. Business Understanding

## Doelstelling
Het doel van dit onderzoek is het vaststellen van de belangrijkste kenmerken (zoals budget
IMDb-scores, en sociale media populariteit) van films die goed presteren, om te begrijpen welke
factoren bijdragen aan success in de filmindustrie.

## Onderzoeksvragen
De volgende onderzoeksvragen worden behandeld in deze opdracht:

1. In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op **Facebook** en **IMDB**?
2. In hoeverre beïnvloedt het slagen voor de Bechdeltest de omzet van een film?
3. In hoeverre is het mogelijk om logische clusters te vinden o.b.v. onder andere budget en omzet? Denk aan blockbusters (hoog budget en hoge omzet), flops (hoog budget en lage omzet) of cultfilms (laag budget en hoge omzet). Je mag extra features toevoegen. 

# 2. Data understanding
De tweede fase in **CRISP-DM** is de **data understanding** dit houdt in:
- Data Collection
- Data Exploration & Analysis

## Data Collection

### Movies dataset
dit is de dataset die we uit deze opdracht hebben meegekregen

### Bechdeltest
De bechdel test is als volgt:
1. It has to have at least two (named) women in it
2. Who talk to each other
3. About something besides a man

https://bechdeltest.com/


In [ ]:
# Dit is de casus dataset
dataset = pd.read_csv('movie.csv')
# dit is de gekozen externe dataset


## Data Exploration & Analysis
text hier om in het kort te beschrijven wat in dit stuk behandeld wordt

### Algemene verkenning
om een eerste blik te krijgen van de Data maken we gebruik van de **head** functie:

In [ ]:
dataset.head()

Om in één blik de kolommen / potentiële features te kunnen zien gebruiken we de **columns** functie:

In [ ]:
dataset.columns

vervolgens gebruiken we de **dtypes** functie om de datatypen van elke kolom te krijgen

In [ ]:
dataset.dtypes

We gebruiken de functie **describe** om een samenvatting te genereren van de numerieke kolommen  

In [ ]:
dataset.describe()

Om de samenvatting van de niet numerieke data te krijgen moeten we dat specificeren in de **describe** functie

In [ ]:
dataset.describe(include='object')

### Target
/// in dit stukje kunnen we om beter inzicht te krijgen in de targetvariabele wat visualisaties neerzetten

### Potential Features
/// hier eerst een korte uitleg waarom we bepaalde features kiezen om te exploren dus al een soort mini hyphothese hoeft niet langer dan een of twee zinnen per feature en dit combineren we met welke meetniveaus deze features hebben


misschien kunnen we hier ook nog kort iets met namen van acteurs/director doen bijv gemiddelde opbrengst per acteur/director dan zien we gelijk of dit waardevolle data is

### Analyse van feature- en targetvariabelen

Te onderzoeken featurevariabelen:  
- movie_facebook_likes
- imdb_score
- budget

Te onderzoeken targetvariabele:  
- gross

In [ ]:
# Kolommen selecteren
data_analyse = dataset['movie_facebook_likes', 'imdb_score', 'budget', 'gross']
data_analyse


In [ ]:
# Centrum- en spreidingsmaten
data_analyse.describe()

In [ ]:
# Uitschieters bepalen
for kolom in data_analyse:

    # Q1, Q3 en IQR bepalen
    q1 = data_analyse[kolom].quantile(0.25)
    q3 = data_analyse[kolom].quantile(0.75)
    iqr = q3 - q1

    # Limieten uitschieters berekenen
    klein_laag = q1 - 1.5 * iqr
    klein_hoog = q3 + 1.5 * iqr
    extreem_laag = q1 - 3 * iqr
    extreem_hoog = q3 + 3 * iqr

    print(data_analyse[kolom].name)

    # Kleine uitschieters (1.5 - 3 IQR)
    kleine_uitschieters = data_analyse[kolom][((data_analyse[kolom] < klein_laag) & (data_analyse[kolom] > extreem_laag)) | ((data_analyse[kolom] > klein_hoog) & (data_analyse[kolom] < extreem_hoog))]
    print(f"Kleine uitschieters: {kleine_uitschieters.count()}")
 
    # Extreme uitschieters (3+ IQR)
    extreme_uitschieters = data_analyse[kolom][(data_analyse[kolom] < extreem_laag) | (data_analyse[kolom] > extreem_hoog)]
    print(f"Extreme uitschieters: {extreme_uitschieters.count()}")

## Data Kwaliteit
Met de functie **isnull** kunnen we controleren hoeveel missende waarden we hebben

In [ ]:
dataset.isnull().sum()

Met de functie **duplicated** checken we of er duplicates in de data zijn. Dit checken we door de waarde van kolommen **movie_title** en **movie_imdb_link** mee te geven aangezien dit eigenlijk de unieke waarden zouden moeten zijn.

In [ ]:
dataset[dataset.duplicated(['movie_title', 'movie_imdb_link'])]

## Waarnemingen

- movie_facebook_likes heeft 930 extreme uitschieters, dat is behoorlijk veel. Zoals eerder benoemd, komt dit waarschijnlijk doordat er veel films zijn zonder facebook pagina, wat geregistreerd wordt als 0 likes.

- imdb_score heeft 0 extreme uitschieters. Dit lijkt ons logisch omdat de imdb score voor een film altijd tussen de 1 en 10 ligt. Er zal dus nooit een waarde van bijvoorbeeld 100 moeten zijn, wat een extreme uitschieter zou zijn.

- De kolommen movie_facebook_likes en imdb_score hebben beide 5043 waarden, maar budget en gross hebben minder waarden (4551 en 4159 waarden).  
We zullen hier een oplossing voor moeten vinden, door bijvoorbeeld de rijen met missende waarden te droppen van de dataset.

- Q1 voor movie_facebook_likes is 0, dit betekent dat ten minste 25% van de films 0 facebook likes hebben. Dit komt waarschijnlijk omdat deze films geen facebook-pagina hebben.

# 3. Data preparation


In [ ]:
#We hebben de link voor de imdb pagina niet nodig maar hierin staat wel de imdb_id die we later kunnen gebruiken om een externe dataset te koppelen.

dataset['movie_imdb_link'] = dataset['movie_imdb_link'].str.slice(start=28, stop=-17)

In [ ]:
#Veranderen van een paar column namen
dataset.rename(columns={'director_name':'director', 'movie_title':'title', 'movie_imdb_link':'imdb_id'}, inplace=True)

# Color Feature

In [ ]:
dataset.describe(include="all")
totaal_films = dataset['color'].count()

kleur_films = dataset[dataset['color'] == 'Color'].shape[0]

zw_films = dataset[dataset['color'] != 'Color'].shape[0]  

percentage_kleur = (kleur_films / totaal_films) * 100
percentage_zw = (zw_films / totaal_films) * 100

print(f"Percentage gekleurde films: {percentage_kleur:.2f}%")
print(f"Percentage zwart-wit films: {percentage_zw:.2f}%")


In [ ]:
#Zoals hierboven zijn bijna 96% van de films in kleur, daarom ervoor gekozen om deze kolom te droppen.
dataset.drop(['color'], axis = 1, inplace=True)


# Duration Feature

In [ ]:
#De duration kan invloed hebben op kijkersbeoordelingen, maar minder op omzet of andere clusters.
dataset.drop(['duration'], axis = 1, inplace=True)

# Actors Like Feature

In [ ]:
#We hebben besloten om de cast_total_facebook_likes te gebruiken, omdat het een beter algemeen beeld geeft.
dataset.drop(['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes'], axis = 1, inplace=True)

# Aspect Ratio Feature

In [ ]:
#Aspect ratio is niet relevant
dataset.drop(['aspect_ratio'], axis = 1, inplace=True)

# Movie Imdb Link Feature

In [ ]:
#Movie Imdb Link is ook niet relevant
#dataset.drop(['movie_imdb_link'], axis = 1, inplace=True)

# Genre Feature

In [ ]:
#We zijn niet van plan om genre op dit moment te gebruiken.
dataset.drop(['genres'], axis = 1, inplace=True)


# Users, Poster en Keywords Feature

In [ ]:
dataset.drop(['num_voted_users', 'facenumber_in_poster','plot_keywords'], axis = 1, inplace=True)



# Reviews Feature

In [ ]:
#Kan interessant zijn voor een populariteitsanalyse, maar minder voor omzet of clustering
dataset.drop(['num_user_for_reviews','num_critic_for_reviews'], axis = 1, inplace=True)



# Content Rating Feature

In [ ]:
#Niet relevant voor ons.
dataset.drop(['content_rating' ], axis = 1, inplace=True)


# Title Year Feature

In [ ]:
dataset.drop(['title_year'], axis = 1, inplace=True)


# Language en Country Feature

In [ ]:
dataset.drop(['language', 'country' ], axis = 1, inplace=True)


#  Gross Target

In [ ]:
#Hier wordt het gemiddelde van de gross waardes die er zijn berekend
gemiddelde_gross = dataset['gross'].mean()
print(f"Gemiddelde gross: ${gemiddelde_gross:.2f}")


In [ ]:

dataset.fillna({"gross" :48468407.53 }, inplace=True)

#  Budget Target

In [ ]:
gemiddelde_budget = dataset['budget'].mean()
print(f"Gemiddelde budget: ${gemiddelde_budget:.2f}")


In [ ]:
dataset.fillna({"budget" :48468407.53 }, inplace=True)

In [ ]:
dataset.head()


# Datatypes